# Prompt Engineering and Chat Completion
This is a simple tutorial that teaches prompt engineering basics and analyzes Twitter data using OpenAI large language models (LLMs).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it securely. This tutorial uses [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLMs predict the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates the quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- openai: call the OpenAI APIs.

In [3]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI
from IPython.display import display, Image, Markdown
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city where the President lives and where you can find major government buildings like the White House and the Capitol Building. If you ever visit, you can also check out many museums and monuments!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

The expression "3##3" does not have a standard mathematical interpretation. If you are referring to a specific operation or notation, please provide more context or clarify the meaning of the symbols.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{sentiment:positive}


## Analyze Twitter data

In [9]:
import json

with open('tweet_text.json', 'r', encoding='utf-8') as f:
    tweet_data = json.load(f)

In [10]:
print("Number of tweets:", len(tweet_data))
print("Sample tweet:", tweet_data[0])

Number of tweets: 182
Sample tweet: {'_id': {'$oid': '68e535897b92a174c5bf0554'}, 'tweet': {'text': '🤖⚠️ En IA generativa, copiar prompts puede alterar modelos y filtrar info sensible. @ZorayaMejia aconseja: crea tus propios prompts, verifica fuentes y educa al equipo. Precaución = profesionalismo.💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9'}}


### Summarization 
- Analyze tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [11]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets cover a wide range of topics related to generative AI, including its impact on various industries, ethical considerations, and practical applications. Key highlights include:

1. **Generative AI in Business and Technology**: Tweets discuss the role of generative AI in transforming creative industries, finance, and local businesses. AI is seen as a tool for innovation, enhancing customer engagement, and streamlining processes.

2. **Ethical and Security Concerns**: There are concerns about the ethical implications of generative AI, such as data privacy, security risks, and the potential for misuse. Deloitte Australia faced issues with AI-generated fake quotes, highlighting the need for ethical AI practices.

3. **AI in Creative and Media Industries**: The use of AI in creating content, such as videos and music, is noted, with discussions on its impact on intellectual property and creative careers. Hollywood has signed agreements on AI use in post-production.

4. **AI Tools and Education**: Tweets mention educational programs and tools for learning about AI, including courses on Retrieval Augmented Generation (RAG) and the use of AI in coding and software development.

5. **Public Perception and Criticism**: Some tweets express skepticism about the widespread use of generative AI, citing environmental concerns and the potential for decreased motivation and engagement among workers.

6. **Events and Innovations**: There are mentions of events like the Generative Gathering and innovations such as AI-powered supply chain analysis and AI in banking.

Overall, the tweets reflect a mix of enthusiasm for the potential of generative AI and caution regarding its ethical and practical implications.

In [12]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

Generative AI is transforming industries, raising ethical concerns, and impacting creativity, coding, and business innovation.

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people learn AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets highlight various aspects of learning AI, emphasizing the importance of creating original prompts, understanding AI's impact on industries, and the need for ethical considerations. They also discuss AI's role in transforming workflows, enhancing creativity, and the potential challenges it poses, such as environmental concerns and skill erosion.

### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [14]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [15]:
for tweet in tweet_data:
    # print(tweet['tweet']['text'])
    flag_help(tweet['tweet']['text'])

===
harassment
Pisses me off when someone makes fun of ppl for having a lot of online friends like ALL MY IRLS ARE HOMOPHOBIC LOVE GENERATIVE AI AND PASS RACIAL SLURS AROUND LIKE A COOKIE JAR 😭🤞


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [16]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ 在生成式人工智能中，复制提示可能会改变模型并泄露敏感信息。@ZorayaMejia 建议：创建你自己的提示，核实来源并教育团队。谨慎=专业。💡 #人工智能 #生成式AI #数字安全 https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: 德勤澳大利亚将部分退还44万美元，因为使用AI生成的虚假报价。 #生成式AI #机器学习 #伦理
RT @jinwashu: 很高兴向大家介绍我们的新工具GAiN，它使用#生成式AI进行基因表达分析，来自放射肿瘤学的@MikeWaters121…
💻 第24天/365天 – JS 函数 学习了如何： 🔹 声明和调用函数 🔹 使用参数和返回值 🔹 编写箭头函数和回调函数 ✨ 函数 = 可重用 + 更简洁的JS代码！ #100天编码 #JavaScript #MERNStack #Web开发 #生成式AI #编码军团 https://t.co/ZRbl6RQXla
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT，这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
转发 @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出了ChatGPT和这些AI视频……


In [17]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 Oh, do pay attention, you simpletons! In the realm of generative AI, copying prompts is akin to playing with matches in a room full of dynamite. It can tamper with models and leak sensitive information. The ever-wise @ZorayaMejia suggests crafting your own prompts, verifying sources, and educating your team. Remember, caution is the hallmark of true professionalism. Now, off you go! 💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9
Oh, how delightfully amusing! It appears that Deloitte Australia is set to return a rather substantial sum of $440,000, all thanks to some mischievous AI-generated fabrications. One must appreciate the irony of technology's little faux pas, don't you think? #GenerativeAi #MachineLearning #Ethics
Ah, splendid! It appears @jinwashu has unveiled a delightful contraption, GAiN, utilizing the wonders of #GenerativeAI for gene expression analysis. Bravo, RadOnc and @MikeWaters121, for your scientific wizardry!
💻 Day 24/365 – JS Functions  
Ah, the del

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract the mentioned people's or organization's names
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [18]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["@ZorayaMejia"]
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Deloitte Australia", "HaywaaWorldwide"]
}
{
  "sentiment": "positive",
  "emotion": "gladness",
  "mentioned": ["jinwashu", "RadOnc", "MikeWaters121"]
}
{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": []
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}


In [19]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Generative AI in creative industries",
    "Ethics and regulation of AI",
    "Generative AI in finance",
    "AI's impact on the environment",
    "AI in education and upskilling",
    "AI in content creation and marketing",
    "AI in software development and coding",
    "AI in entertainment and media",
    "AI in business and enterprise solutions",
    "AI in healthcare and data interpretation"
  ]
}


### Expanding with multiple prompts 
- Identify topics
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [20]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative, or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 182/182 [02:46<00:00,  1.09it/s]


In [21]:
print(analysis_result)

['{"sentiment": "neutral", "emotion": "caution", "mentioned": ["@ZorayaMejia"]}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["HaywaaWorldwide", "Deloitte Australia"]\n}', '{"sentiment": "positive", "emotion": "gladness", "mentioned": ["jinwashu", "MikeWaters121"]}', '{\n  "sentiment": "positive",\n  "emotion": "enthusiasm",\n  "mentioned": []\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDa

In [22]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis result delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets mentioned by people or organizations;
                                        step 2 {delimiter} identify the common sentiments and emotions for each mentioned person or organization;
                                        step 3 {delimiter} Organize the result in a json document with keys <people or organization name> 
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "@ZorayaMejia": {
    "count": 1,
    "sentiments": ["neutral"],
    "emotions": ["caution"]
  },
  "HaywaaWorldwide": {
    "count": 2,
    "sentiments": ["neutral", "negative"],
    "emotions": ["informative", "concern"]
  },
  "Deloitte Australia": {
    "count": 2,
    "sentiments": ["neutral", "neutral"],
    "emotions": ["informative", "none"]
  },
  "jinwashu": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "MikeWaters121": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "AmandaDannielle": {
    "count": 38,
    "sentiments": ["neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neut

## Create a chatbot

In [23]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chatbot that answers user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people or organization name in the {delimiter}{analysis_summary}{delimiter}, report the corresponding sentiment and emotion
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [24]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    display(Markdown(f"Chatbot: {reply}"))

You:  how to learn ai


Chatbot: There are many ways to learn AI, depending on your current skill level and learning style. Here's a general guide to get started:

1. **Basics of Programming:**
   - Learn programming languages commonly used in AI, such as Python or R.

2. **Mathematics:**
   - Understand the mathematical concepts that underpin AI, including algebra, calculus, statistics, and probability.

3. **Online Courses and Tutorials:**
   - Platforms like Coursera, edX, and Udacity offer specialized AI courses.
   - Explore free resources on YouTube and other free learning websites.

4. **Books and Articles:**
   - Read books focused on AI and machine learning to build foundational knowledge.

5. **Join AI Communities:**
   - Participate in AI forums, attend meetups, and follow AI experts on social media to stay updated and engage with a community.
   
6. **Projects and Hands-On Practice:**
   - Work on AI projects to apply what you've learned and build a portfolio.
   - Use platforms like Kaggle to practice with real datasets.

7. **Get Familiar with AI Tools:**
   - Learn to use AI and machine learning frameworks like TensorFlow, PyTorch, and Scikit-learn.

8. **Keep Up with Trends:**
   - Stay informed about the latest advancements and ethical considerations in AI. 

9. **Advanced Learning:**
   - Consider university courses or online certifications for more structured learning.

Start by setting clear learning goals and gradually build your expertise in each area.

You:  how do people talk about chatgpt


Chatbot: Based on the tweets, discussions around ChatGPT often have a neutral sentiment, focusing on informative and cautionary aspects. Here are some key themes observed:

1. **Caution and Concerns:** Many tweets express caution about using ChatGPT, highlighting the potential risks and advising users on which types of AI to stay away from.

2. **Informative Sharing:** Users frequently share informative content about ChatGPT, discussing its uses and capabilities in various contexts.

3. **Neutral Sentiment:** Overall, the sentiment remains neutral, with people neither expressing strong positive nor negative opinions but rather focusing on sharing information and precautions. 

It's clear that there is an emphasis on spreading awareness about responsible and informed use of ChatGPT.

You:  exit


Chatbot: Goodbye!


## 🧠 Try It Yourself: Prompt Engineering Practice
👉 Be sure to **run your own cells** and show the outputs before uploading the notebook to GitHub.

1. **Summarize your tweets differently**  
   Change your system message to summarize tweets from a new perspective —  
   for example:  
   *“Summarize what people say about AI in education.”*  
   or  
   *“Summarize how people feel about the election.”*

In [28]:
messages = [
    {"role": "system", "content": f"""Summarize what people say about AI in education"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The discussion around AI in education, particularly generative AI, is multifaceted. Some emphasize the potential of AI to transform learning and business processes, suggesting that AI-powered learning is an investment in the future. Others highlight the importance of creating original prompts and verifying sources to maintain professionalism and security. Concerns are raised about the ethical implications and environmental impact of AI, with some advising caution in its use. There is also a debate on whether AI serves as a beneficial tool or a crutch that might erode essential skills. Additionally, the role of AI in creative industries and its regulation and ethics are frequently discussed, reflecting both optimism and skepticism about its integration into various sectors.

2. **Translate tweets**  
   Write a prompt that translates 10 tweets into another language,  
   such as Chinese, Spanish, or French.


In [29]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into French"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ En IA générative, copier des invites peut altérer les modèles et divulguer des informations sensibles. @ZorayaMejia conseille : créez vos propres invites, vérifiez les sources et éduquez l'équipe. Prudence = professionnalisme.💡 #IA #GenerativeAI #SécuritéNumérique https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: Deloitte Australie remboursera partiellement 440 000 $ en raison de fausses citations générées par l'IA. #GenerativeAi #MachineLearning #Eth…
RT @jinwashu : Heureux de vous présenter notre nouvel outil GAiN qui utilise l'#IAgénérative pour l'analyse de l'expression génique, de RadOnc @MikeWaters121…
💻 Jour 24/365 – Fonctions JS
J'ai appris à :
🔹 Déclarer et appeler des fonctions
🔹 Utiliser des paramètres et des retours
🔹 Écrire des fonctions fléchées et de rappel

✨ Fonctions = code JS réutilisable + plus propre !

#100DaysOfCode #JavaScript #MERNStack #WebDev #GenerativeAI #coderarmy https://t.co/ZRbl6RQXla
RT @AmandaDannielle : Les gens n'arrêtent pas de vous dire quels types

3. **Simplify or rewrite tweets**  
   Ask the model to rewrite 10 tweets in a more polite, formal, or funny tone.

In [30]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Obama """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 In the realm of generative AI, replicating prompts can impact models and potentially expose sensitive information. @ZorayaMejia offers wise counsel: develop your own prompts, verify your sources, and educate your team. Caution is the hallmark of professionalism. Let's stay vigilant. 💡 #AI #GenerativeAI #DigitalSecurity 
RT @HaywaaWorldwide: Deloitte Australia has taken a commendable step by agreeing to partially refund $440,000 due to AI-generated fake quotes. This is a reminder of the importance of ethics and responsibility in the age of #GenerativeAI and #MachineLearning.
I'm thrilled to share the exciting news about the innovative tool, GAiN, which harnesses the power of #GenerativeAI for gene expression analysis. A big congratulations to the team at RadOnc and @MikeWaters121 for their groundbreaking work. Let's continue to push the boundaries of science and technology together.
 Today marks Day 24 of my journey through the world of JavaScript. I've had the opportunity to delve int

4. **Find keywords**  
   Design a prompt that extracts the top five most common keywords  
   from your tweet text and returns them as a list.

In [31]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify top 5 key words, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "keywords": [
    "GenerativeAI",
    "AI",
    "MachineLearning",
    "Innovation",
    "CreativeIndustries"
  ]
}


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
